## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# Check types of columns
model_data.dtypes

Angaus          int64
SegSumT       float64
SegTSeas      float64
SegLowFlow    float64
DSDist        float64
DSMaxSlope    float64
USAvgT        float64
USRainDays    float64
USSlope       float64
USNative      float64
DSDam           int64
Method         object
LocSed        float64
dtype: object

In [3]:

# encode categorical variable "Method"
label_encoder = LabelEncoder()

model_data['Method'] =  label_encoder.fit_transform(model_data['Method'])

# Check it worked
model_data.head()


,Angaus,SegSumT,SegTSeas,SegLowFlow,DSDist,DSMaxSlope,USAvgT,USRainDays,USSlope,USNative,DSDam,Method,LocSed
0,0,16.0,-0.10,1.036,50.20,0.57,0.09,2.470,9.8,0.81,0,0,4.8
1,1,18.7,1.51,1.003,132.53,1.15,0.20,1.153,8.3,0.34,0,0,2.0
2,0,18.3,0.37,1.001,107.44,0.57,0.49,0.847,0.4,0.00,0,3,1.0
3,0,16.7,-3.80,1.000,166.82,1.72,0.90,0.210,0.4,0.22,1,0,4.0
4,1,17.2,0.33,1.005,3.95,1.15,-1.20,1.980,21.9,0.96,0,0,4.7


In [4]:
# get target variable and features
y = model_data['Angaus']

X = model_data.drop('Angaus', axis = 1)

In [5]:
# Split data for training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=808)

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [8]:
# intialize xgb classifer
xgb = XGBClassifier(n_estimators = 1000,
                    learning_rate = 0.1,
                    eval_metric = "logloss",
                    early_stopping_rounds = 50,
                    random_state = 808
                    )

# train xgb model
xgb.fit(X_train, y_train, eval_set=[(X_val,y_val)])


# Get and print the best number of trees
best_trees = xgb.best_iteration  
print(f"Best number of trees: {best_trees}")

[0]	validation_0-logloss:0.51354
[1]	validation_0-logloss:0.48803
[2]	validation_0-logloss:0.46738
[3]	validation_0-logloss:0.45126
[4]	validation_0-logloss:0.43766
[5]	validation_0-logloss:0.42555
[6]	validation_0-logloss:0.41728
[7]	validation_0-logloss:0.40769
[8]	validation_0-logloss:0.40082
[9]	validation_0-logloss:0.39562
[10]	validation_0-logloss:0.39263
[11]	validation_0-logloss:0.38984
[12]	validation_0-logloss:0.38489
[13]	validation_0-logloss:0.38327
[14]	validation_0-logloss:0.37803
[15]	validation_0-logloss:0.37652
[16]	validation_0-logloss:0.37337
[17]	validation_0-logloss:0.37156
[18]	validation_0-logloss:0.36987
[19]	validation_0-logloss:0.36868
[20]	validation_0-logloss:0.36809
[21]	validation_0-logloss:0.36721
[22]	validation_0-logloss:0.36763
[23]	validation_0-logloss:0.36570
[24]	validation_0-logloss:0.36577
[25]	validation_0-logloss:0.36666
[26]	validation_0-logloss:0.36559
[27]	validation_0-logloss:0.36552
[28]	validation_0-logloss:0.36422
[29]	validation_0-loglos

### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [9]:
xgb2 = XGBClassifier(
    n_estimators=best_trees,
    learning_rate=0.1,
    eval_metric="logloss",
    early_stopping_rounds=50,
    random_state=808
)

# Fit model with early stopping
xgb2.fit(
    X_train, y_train, eval_set=[(X_val, y_val)]
)

param_grid = {
    "learning_rate":uniform(loc=0.01, scale=0.30)
}

rs = RandomizedSearchCV(xgb2, param_distributions=param_grid,
                         return_train_score=True, n_iter=20,
                            random_state=808)

rs.fit(X_train, y_train, eval_set=[(X_val, y_val)])

[0]	validation_0-logloss:0.51354
[1]	validation_0-logloss:0.48803
[2]	validation_0-logloss:0.46738
[3]	validation_0-logloss:0.45126
[4]	validation_0-logloss:0.43766
[5]	validation_0-logloss:0.42555
[6]	validation_0-logloss:0.41728
[7]	validation_0-logloss:0.40769
[8]	validation_0-logloss:0.40082
[9]	validation_0-logloss:0.39562
[10]	validation_0-logloss:0.39263
[11]	validation_0-logloss:0.38984
[12]	validation_0-logloss:0.38489
[13]	validation_0-logloss:0.38327
[14]	validation_0-logloss:0.37803
[15]	validation_0-logloss:0.37652
[16]	validation_0-logloss:0.37337
[17]	validation_0-logloss:0.37156
[18]	validation_0-logloss:0.36987
[19]	validation_0-logloss:0.36868
[20]	validation_0-logloss:0.36809
[21]	validation_0-logloss:0.36721
[22]	validation_0-logloss:0.36763
[23]	validation_0-logloss:0.36570
[24]	validation_0-logloss:0.36577
[25]	validation_0-logloss:0.36666
[26]	validation_0-logloss:0.36559
[27]	validation_0-logloss:0.36552
[28]	validation_0-logloss:0.36422
[0]	validation_0-logloss

RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None, n_estimators=29,
                                           n_jobs=None, num_parallel_tree=None,
                                           random_state=808, ...),
                   n_iter=20,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fccb8ce3370>},
                   random_state=808, return_train_score=True)

In [10]:
# Print best number of learners
best_lr = rs.best_params_['learning_rate']
print(f"Best learning rate: {best_lr:.4f}")

Best learning rate: 0.2429


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [14]:
xgb3 = XGBClassifier(
    n_estimators = best_trees,
    learning_rate = best_lr,
    eval_metric = "logloss",
    early_stopping_rounds = 50,
    random_state = 808
)

# Fit model with early stopping
xgb3.fit(
    X_train, y_train, eval_set = [(X_val, y_val)])

param_dict = {
    "max_depth": randint(3, 7),
    "min_child_weight": randint(1, 10),
    "gamma": uniform(loc = 0.05, scale = 0.05) 
}

rs2 = RandomizedSearchCV(xgb2, param_distributions=param_dict,
                         return_train_score=True, n_iter=20,
                            random_state=808)

rs2.fit(X_train, y_train, eval_set=[(X_val, y_val)])

[0]	validation_0-logloss:0.47507
[1]	validation_0-logloss:0.43369
[2]	validation_0-logloss:0.42154
[3]	validation_0-logloss:0.40527
[4]	validation_0-logloss:0.39260
[5]	validation_0-logloss:0.38723
[6]	validation_0-logloss:0.37547
[7]	validation_0-logloss:0.37376
[8]	validation_0-logloss:0.37151
[9]	validation_0-logloss:0.37117
[10]	validation_0-logloss:0.36473
[11]	validation_0-logloss:0.36333
[12]	validation_0-logloss:0.36329
[13]	validation_0-logloss:0.36659
[14]	validation_0-logloss:0.36495
[15]	validation_0-logloss:0.36629
[16]	validation_0-logloss:0.36805
[17]	validation_0-logloss:0.36624
[18]	validation_0-logloss:0.37117
[19]	validation_0-logloss:0.37227
[20]	validation_0-logloss:0.37188
[21]	validation_0-logloss:0.37163
[22]	validation_0-logloss:0.36787
[23]	validation_0-logloss:0.36779
[24]	validation_0-logloss:0.37083
[25]	validation_0-logloss:0.37064
[26]	validation_0-logloss:0.37322
[27]	validation_0-logloss:0.37313
[28]	validation_0-logloss:0.37332
[0]	validation_0-logloss

RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0...
                                           random_state=808, ...),
                   n_iter=20,
                   param_distributions={'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fcc0f533010>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fccb90aba30>,
                                        'min_child_weight': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fccb8e6b4f0>},
                   random_state=808, return_train_score=True)

In [15]:
# Print best parameters
best_tree_params = rs2.best_params_
print(f"Best tree parameters: {best_tree_params}")

Best tree parameters: {'gamma': 0.06673574366507203, 'max_depth': 5, 'min_child_weight': 6}


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [16]:
# Initialize fourth XGB model
xgb4 = XGBClassifier(
    n_estimators=best_trees,
    learning_rate=best_lr,
    **best_tree_params, 
    early_stopping_rounds=50, 
    eval_metric="logloss", 
    random_state=808)

# Fit model
xgb4.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Third param dist
param_dist3 = {
    "subsample": uniform(0.5, 0.5),
    "colsample_bytree": uniform(0.5, 0.5) 
}

# Set up RandomizedSearchCV
rs3 = RandomizedSearchCV(
    xgb4, param_dist3, n_iter=20, scoring='accuracy', 
    cv=3, verbose=False, random_state=808
)

# Run random search
rs3.fit(X_train, y_train, eval_set=[(X_val, y_val)])

[0]	validation_0-logloss:0.48425
[1]	validation_0-logloss:0.44631
[2]	validation_0-logloss:0.42356
[3]	validation_0-logloss:0.40968
[4]	validation_0-logloss:0.39749
[5]	validation_0-logloss:0.39123
[6]	validation_0-logloss:0.38743
[7]	validation_0-logloss:0.38178
[8]	validation_0-logloss:0.37918
[9]	validation_0-logloss:0.37695
[10]	validation_0-logloss:0.37838
[11]	validation_0-logloss:0.37495
[12]	validation_0-logloss:0.37113
[13]	validation_0-logloss:0.36943
[14]	validation_0-logloss:0.36987
[15]	validation_0-logloss:0.36847
[16]	validation_0-logloss:0.36100
[17]	validation_0-logloss:0.36066
[18]	validation_0-logloss:0.36189
[19]	validation_0-logloss:0.35902
[20]	validation_0-logloss:0.35419
[21]	validation_0-logloss:0.35568
[22]	validation_0-logloss:0.35513
[23]	validation_0-logloss:0.35478
[24]	validation_0-logloss:0.35690
[25]	validation_0-logloss:0.35833
[26]	validation_0-logloss:0.35950
[27]	validation_0-logloss:0.35796
[28]	validation_0-logloss:0.35503
[0]	validation_0-logloss

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           gamma=0.06673574366507203,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=...
                                           multi_strategy=None, n_estimators=29,
                                           n_jobs=None, num_parallel_tree=None,
                                           random_state=808, ...),
                   n_iter=20,
                   param_distributions={'colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fccb90b1ba0>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fccb908bcd0>},
                   random_state=808, scoring='accuracy', verbose=False)

In [17]:
# Print best stochastic parameters
best_stoch_params = rs3.best_params_
print(f"Best stochastic parameters: {best_stoch_params}")

Best stochastic parameters: {'colsample_bytree': 0.6673574366507202, 'subsample': 0.8046025228990683}


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [18]:
# Check column types
eval_data.dtypes

Angaus_obs      int64
SegSumT       float64
SegTSeas      float64
SegLowFlow    float64
DSDist        float64
DSMaxSlope    float64
USAvgT        float64
USRainDays    float64
USSlope       float64
USNative      float64
DSDam           int64
Method         object
LocSed        float64
dtype: object

In [19]:
eval_data['Method'] =  label_encoder.fit_transform(eval_data['Method'])


# Assign features
X_eval= eval_data.drop('Angaus_obs', axis=1)
y_eval = eval_data['Angaus_obs']


# Initialize fifth XGB model
xgb5 = XGBClassifier(
    n_estimators=best_trees,
    learning_rate=best_lr,
    **best_tree_params, 
    **best_stoch_params, 
    early_stopping_rounds=50, 
    eval_metric="logloss", 
    random_state=808)

# Fit model
xgb5.fit(X_train, y_train, eval_set=[(X_val, y_val)])
preds = xgb5.predict(X_eval)

[0]	validation_0-logloss:0.51710
[1]	validation_0-logloss:0.49430
[2]	validation_0-logloss:0.45417
[3]	validation_0-logloss:0.42519
[4]	validation_0-logloss:0.41553
[5]	validation_0-logloss:0.40202
[6]	validation_0-logloss:0.39105
[7]	validation_0-logloss:0.38674
[8]	validation_0-logloss:0.37799
[9]	validation_0-logloss:0.37770
[10]	validation_0-logloss:0.37751
[11]	validation_0-logloss:0.37943
[12]	validation_0-logloss:0.37849
[13]	validation_0-logloss:0.38485
[14]	validation_0-logloss:0.38101
[15]	validation_0-logloss:0.37368
[16]	validation_0-logloss:0.37476
[17]	validation_0-logloss:0.37427
[18]	validation_0-logloss:0.37276
[19]	validation_0-logloss:0.37213
[20]	validation_0-logloss:0.36469
[21]	validation_0-logloss:0.36344
[22]	validation_0-logloss:0.35975
[23]	validation_0-logloss:0.35719
[24]	validation_0-logloss:0.35909
[25]	validation_0-logloss:0.36151
[26]	validation_0-logloss:0.35969
[27]	validation_0-logloss:0.36040
[28]	validation_0-logloss:0.35723


### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [20]:
# Get probability predictions 
pred_probs = xgb5.predict_proba(X_eval)[:, 1]

# Calculate ROC score
auc_score = roc_auc_score(y_eval, pred_probs)
print(f"AUC Score: {auc_score:.4f}")

AUC Score: 0.8734


In [21]:
# Get feature importance
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': xgb5.feature_importances_})

# Sort by importance
feature_importance = feature_importance.sort_values(by="Importance", ascending=False)
feature_importance

,Feature,Importance
8,USNative,0.162220
0,SegSumT,0.155243
10,Method,0.110354
4,DSMaxSlope,0.108152
6,USRainDays,0.082682
1,SegTSeas,0.077976
5,USAvgT,0.074422
3,DSDist,0.068250
7,USSlope,0.057833
2,SegLowFlow,0.056972


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

My model performance had a very similar result to the paper's. My top three features (ranked by performance) were the same, but the order was slightly different. Instead of having the order be: SegSumT, USNative, and Method, mine was USNative, SegSumT, and Method. In addition to this, my AUC was also pretty similar. Mine was 0.8734 and theirs was 0.869. 